In [ ]:
import sys
from os import cpu_count
sys.path.append("../")
from mappings import hubbard_mapping_helper as mapper
from mappings import jw_9_mode as jw_9
from mappings import lw_9_mode as lw_9
from mappings import gse_9_mode as gse_9
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc_file_defaults()
plt.style.use('ggplot')
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
import numpy as np
from simulator.logical_vector_simulator import logical_state_simulation
from simulator.noise import single_qubit_depolarizing, parallel_single_qubit_depolarizing, noiseless
from simulator.utils import check_mapping, fidelity
import pickle
from random import choices, seed

### Notebook for noisy random circuit fidelity experiments on systems of 9 fermionic modes

To recreate plots or re-do previous experiments, load the pickled data file and use the old experimental parameters

#### Optional load in old data

In [ ]:
"""with open(f'9moderandom_circuit_data.pkl', "rb") as f:
    data_dict = pickle.load(f)
labels = data_dict['labels']
colors = data_dict['colors']
seeds = data_dict['seeds']
rotations = data_dict['rotations']
averaged_mapping_depths_combined = data_dict['averaged_mapping_depths_combined']
averaged_fidelities = data_dict['averaged_fidelities']
averaged_traces = data_dict['averaged_traces']
sampling_increase = data_dict['sampling_increase']
print(data_dict['random_circ_instances'])"""

In [ ]:
rotations = [j for j in range(200)]
mappings = ['jw_9','lw_9','gse_9']
random_circ_instances = 100
mapping_shots = {
    'jw_9': 100,
    'lw_9': 100,
    'gse_9':100
}
shots = 1

stat_noise_rate = 0
gate_noise_rate = 0.01
if stat_noise_rate==0.0:
    stat_noise_model = noiseless()
else:
    stat_noise_model = single_qubit_depolarizing(stat_noise_rate)
if gate_noise_rate==0.0:
    gate_noise_model = noiseless()
else:
    #gate_noise_model = single_qubit_depolarizing(gate_noise_rate)
    gate_noise_model = parallel_single_qubit_depolarizing(gate_noise_rate)

cpus = cpu_count()-2

In [ ]:
mapping_stabs = {
    'jw_9': [],
    'lw_9': lw_9.stabs,
    'gse_9': gse_9.stabs
}
mapping_Aops = {
    'jw_9': jw_9.Aop,
    'lw_9': lw_9.Aop,
    'gse_9': gse_9.Aop
}
mapping_Bops = {
    'jw_9': jw_9.Bop,
    'lw_9': lw_9.Bop,
    'gse_9': gse_9.Bop
}
mapping_qubits = {
    'jw_9': 9,
    'lw_9': 11,
    'gse_9': 14}
mapping_syndromes = {}
mapping_fixed_ops = {}
for mapping in mappings:
    mapping_fixed_ops[mapping] = []
    for j in range(9):
        mapping_fixed_ops[mapping].append(mapping_Bops[mapping][j])
    # Remove the last one for the mappings where global parity is included in stabilizers
    if mapping_qubits[mapping] - len(mapping_stabs[mapping]) < 9:
        mapping_fixed_ops[mapping] = mapping_fixed_ops[mapping][:-1]
for mapping in mappings:
    print(mapping)
    check_mapping(mapping_Aops[mapping],mapping_Bops[mapping], mapping_stabs[mapping])


In [ ]:
print(type(mapping_stabs['lw_9'][0]))

In [ ]:
avg_Aop_weights = {}
avg_Bop_weights = {}
avg_stab_weights = {}

def op_weight(op):
    if type(op) is list:
        total_weight = 0
        for j in range(len(op)):
            total_weight += len(list(op[j].terms.items())[0][0])
        return total_weight
    if type(op) is dict:
        op_list = list(op.items())
        total_weight = 0
        for j in range(len(op_list)):
            total_weight += len(list(op_list[j][1].terms.items())[0][0])
        return total_weight
    else:
        return len(list(op.terms.items())[0][0])

for mapping in mappings:
    Aop_list = list(mapping_Aops[mapping].items())
    summed_weight = 0
    for j in range(len(Aop_list)):
        summed_weight += op_weight(Aop_list[j][1])
    avg_Aop_weights[mapping] = summed_weight/len(Aop_list)

    Bop_list = list(mapping_Bops[mapping].items())
    summed_weight = 0
    for j in range(len(Bop_list)):
        summed_weight += op_weight(Bop_list[j][1])
    avg_Bop_weights[mapping] = summed_weight/len(Bop_list)

    summed_weight = 0
    for j in range(len(mapping_stabs[mapping])):
        summed_weight += op_weight(mapping_stabs[mapping][j])
    if len(mapping_stabs[mapping])>0:
        avg_stab_weights[mapping] = summed_weight/len(mapping_stabs[mapping])
    else:
        avg_stab_weights[mapping] = 0

In [ ]:
print(avg_Aop_weights)
print(avg_Bop_weights)
print(avg_stab_weights)

In [ ]:
avg_gen_weight = {}
avg_2_qubit_gate_count = {}
for mapping in mappings:
    rotation_set = [
        mapping_Aops[mapping][(0,1)],
        mapping_Aops[mapping][(1,2)],
        mapping_Aops[mapping][(3,4)],
        mapping_Aops[mapping][(4,5)],
        mapping_Aops[mapping][(6,7)],
        mapping_Aops[mapping][(7,8)],
        mapping_Aops[mapping][(0,3)],
        mapping_Aops[mapping][(3,6)],
        mapping_Aops[mapping][(1,4)],
        mapping_Aops[mapping][(4,7)],
        mapping_Aops[mapping][(2,5)],
        mapping_Aops[mapping][(5,8)],
        mapping_Bops[mapping][0]*mapping_Bops[mapping][1],
        mapping_Bops[mapping][1]*mapping_Bops[mapping][2],
        mapping_Bops[mapping][3]*mapping_Bops[mapping][4],
        mapping_Bops[mapping][4]*mapping_Bops[mapping][5],
        mapping_Bops[mapping][6]*mapping_Bops[mapping][7],
        mapping_Bops[mapping][7]*mapping_Bops[mapping][8],
        mapping_Bops[mapping][0]*mapping_Bops[mapping][3],
        mapping_Bops[mapping][3]*mapping_Bops[mapping][6],
        mapping_Bops[mapping][1]*mapping_Bops[mapping][4],
        mapping_Bops[mapping][4]*mapping_Bops[mapping][7],
        mapping_Bops[mapping][2]*mapping_Bops[mapping][5],
        mapping_Bops[mapping][5]*mapping_Bops[mapping][8],
    ]
    for j in range(9):
        rotation_set.append(mapping_Bops[mapping][j])
    avg_gen_weight[mapping] = 0
    avg_2_qubit_gate_count[mapping] = 0
    for k in range(len(rotation_set)):
        avg_gen_weight[mapping] += op_weight(rotation_set[k])
        avg_2_qubit_gate_count[mapping] += 2*(op_weight(rotation_set[k])-1)
    avg_gen_weight[mapping] = avg_gen_weight[mapping]/len(rotation_set)
    avg_2_qubit_gate_count[mapping] = avg_2_qubit_gate_count[mapping]/len(rotation_set)
print(avg_gen_weight)
print(avg_2_qubit_gate_count)

In [ ]:
def fidelity_decay(avg_2_qubit_gate_count,rotations,gate_noise_rate):
    return (1 - 3 * gate_noise_rate / 4)**(2*avg_2_qubit_gate_count*rotations )
print(fidelity_decay(avg_2_qubit_gate_count['gse_9'],100,0.01))

In [ ]:
def error_per_rot(two_q_gates,gate_noise_rate):
    expect_total = 0
    norm_total = 0
    for j in range(int(2*np.ceil(two_q_gates))):
        expect_total += j * (1-gate_noise_rate)**j * (gate_noise_rate)**(2*np.ceil(two_q_gates) - j)
        norm_total += (1-gate_noise_rate)**j * (gate_noise_rate)**(2*np.ceil(two_q_gates) - j)
    return expect_total#/norm_total

In [ ]:
mapping_rots_per_error = {}
mapping_errors_per_rot = {}
gate_noise_rate= 0.02
for mapping in mappings:
    mapping_errors_per_rot[mapping] = error_per_rot(avg_2_qubit_gate_count[mapping],gate_noise_rate)
    mapping_rots_per_error[mapping] = 1/mapping_errors_per_rot[mapping]
print(mapping_rots_per_error)
print(mapping_errors_per_rot)

In [ ]:
def circuit_builder_random_rotations(mapping, num_rotations, rand_seed = None):
    if rand_seed is not None:
        seed(a=rand_seed)
    rotation_set = [
        mapping_Aops[mapping][(0,1)],
        mapping_Aops[mapping][(1,2)],
        mapping_Aops[mapping][(3,4)],
        mapping_Aops[mapping][(4,5)],
        mapping_Aops[mapping][(6,7)],
        mapping_Aops[mapping][(7,8)],
        mapping_Aops[mapping][(0,3)],
        mapping_Aops[mapping][(3,6)],
        mapping_Aops[mapping][(1,4)],
        mapping_Aops[mapping][(4,7)],
        mapping_Aops[mapping][(2,5)],
        mapping_Aops[mapping][(5,8)],
        mapping_Bops[mapping][0]*mapping_Bops[mapping][1],
        mapping_Bops[mapping][1]*mapping_Bops[mapping][2],
        mapping_Bops[mapping][3]*mapping_Bops[mapping][4],
        mapping_Bops[mapping][4]*mapping_Bops[mapping][5],
        mapping_Bops[mapping][6]*mapping_Bops[mapping][7],
        mapping_Bops[mapping][7]*mapping_Bops[mapping][8],
        mapping_Bops[mapping][0]*mapping_Bops[mapping][3],
        mapping_Bops[mapping][3]*mapping_Bops[mapping][6],
        mapping_Bops[mapping][1]*mapping_Bops[mapping][4],
        mapping_Bops[mapping][4]*mapping_Bops[mapping][7],
        mapping_Bops[mapping][2]*mapping_Bops[mapping][5],
        mapping_Bops[mapping][5]*mapping_Bops[mapping][8],
    ]
    for j in range(9):
        rotation_set.append(mapping_Bops[mapping][j])
    circuit = choices(rotation_set, k=num_rotations)
    for j in range(num_rotations):
        circuit[j] = np.random.rand() * circuit[j]
    return circuit

In [ ]:
def depth_and_fidelity(
    phys_qubits, 
    schedule, 
    stabs, 
    init_fixed_ops, 
    stat_noise_model, 
    gate_noise_model, 
    shots,
    syndromes=[],
    cpus=1):
    """
    """
    if isinstance(syndromes,list):
        if len(syndromes)==0:
            blocks_type='code'
        else:
            blocks_type='custom'
    elif isinstance(syndromes,str):
        if syndromes =='all':
            blocks_type='all'
        if syndromes == 'code':
            blocks_type='code'
    
    block_entries = 0
    zero_block_entries = 0
    attempts = 0
    while block_entries == 0:
        attempts +=1
        (state, block_entries, ideal_state, depth, data)  = logical_state_simulation(
            stabilizers = stabs,
            n_phys_qubits = phys_qubits,
            rounds = shots,
            stat_noise = stat_noise_model,
            gate_noise = gate_noise_model,
            phys_circuit = schedule,
            logical_operators = init_fixed_ops,
            d_matrix_blocks = blocks_type,
            block_numbers = syndromes,
            num_processes=cpus
            )
        if block_entries==0:
            print('------Zero block entries------')
            zero_block_entries +=1
    updated_stabs = data[0]
    qubit_order = data[1]
    fixed_positions = data[2]
    fixed_ops = data[3]
    other_ops = data[4]

    if blocks_type=='custom':
        codespace_state = state[tuple(np.asarray([0 for j in range(len(stabs))]))]
        np.real(np.trace(codespace_state)) * block_entries/(attempts*mapping_shots[mapping])
        codespace_trace = np.real(np.trace(codespace_state))
        codespace_state = (1/codespace_trace)*codespace_state
    else:
        codespace_state = state
        codespace_trace = np.real(np.trace(codespace_state))
        codespace_state = (1/codespace_trace)*codespace_state
    codespace_fidelity = fidelity(codespace_state,ideal_state)

    ideal_DM = np.kron(ideal_state, np.conj(np.transpose(ideal_state))).reshape((ideal_state.shape[0],ideal_state.shape[0]))

    ideal_DM_diag = np.diag(ideal_DM)
    noisy_state_diag = np.diag(codespace_state)

    lin_XEB = ideal_state.shape[0] * np.sum(np.multiply(ideal_DM_diag , noisy_state_diag)) - 1
    probs_TVD = 0.5 * np.sum( np.abs ( ideal_DM_diag - noisy_state_diag) )

    return depth, codespace_fidelity, codespace_trace * (block_entries/(attempts*shots)), lin_XEB, probs_TVD, zero_block_entries

In [ ]:
all_prep_depths = np.zeros((len(rotations),random_circ_instances,len(mappings)))
all_circ_depths = np.zeros((len(rotations),random_circ_instances,len(mappings)))
all_fidelities = np.zeros((len(rotations),random_circ_instances,len(mappings)))
all_traces = np.zeros((len(rotations),random_circ_instances,len(mappings)))
all_TVDs = np.zeros((len(rotations),random_circ_instances,len(mappings)))
all_lin_XEBs = np.zeros((len(rotations),random_circ_instances,len(mappings)))
all_gate_counts = np.zeros((len(rotations),random_circ_instances,len(mappings)))

In [ ]:
zero_block_entries_occurances = 0
seeds = [0 for j in range(len(rotations))]
for j in range(len(rotations)):
    seeds[j] = np.random.rand(random_circ_instances)
    for k in range(random_circ_instances):
        for l in range(len(mappings)):
            print(f'rotation set: {j+1}/{len(rotations)}, circuit instance: {k+1}/{random_circ_instances}, {mappings[l]} (mapping: {l+1}/{len(mappings)}), zero block entries: {zero_block_entries_occurances}')
            schedule = circuit_builder_random_rotations(mappings[l], rotations[j], rand_seed = seeds[j][k])
            if mappings[l] in mappings:
                schedule = schedule + mapping_stabs[mappings[l]]
            all_gate_counts[j,k,l] += 2*op_weight(mapping_fixed_ops[mappings[l]]) - 2*len(mapping_fixed_ops[mappings[l]]) + 2*op_weight(schedule) - 2*len(schedule)
            depth, all_fidelities[j,k,l], all_traces[j,k,l],  all_lin_XEBs[j,k,l], all_TVDs[j,k,l], zero_block_entries =  depth_and_fidelity(
                mapping_qubits[mappings[l]], 
                schedule, 
                mapping_stabs[mappings[l]], 
                mapping_fixed_ops[mappings[l]], 
                stat_noise_model, 
                gate_noise_model, 
                mapping_shots[mappings[l]],
                syndromes=[],
                cpus=cpus)
            all_prep_depths[j,k,l] = depth[0]
            all_circ_depths[j,k,l] = depth[1]
            zero_block_entries_occurances += zero_block_entries

In [ ]:
averaged_fidelities = np.zeros((len(mappings),len(rotations)))
averaged_traces = np.zeros((len(mappings),len(rotations)))
averaged_TVDs = np.zeros((len(mappings),len(rotations)))
averaged_lin_XEBs = np.zeros((len(mappings),len(rotations)))
averaged_gate_counts = np.zeros((len(mappings),len(rotations)))
# Take averages of over the random circuit instances
for j in range(len(mappings)):
    for k in range(len(rotations)):
        for l in range(random_circ_instances):
            averaged_fidelities[j,k] += all_fidelities[k,l,j]
            #averaged_traces[j,k] += all_traces[k,l,j]
            #averaged_lin_XEBs[j,k] += all_lin_XEBs[k,l,j]
            #averaged_TVDs[j,k] += all_TVDs[k,l,j]
            averaged_gate_counts[j,k] += all_gate_counts[k,l,j]
        #averaged_fidelities[j,k] = (1/random_circ_instances)*averaged_fidelities[j,k]
        #averaged_traces[j,k] = (1/random_circ_instances)*averaged_traces[j,k]
        #averaged_lin_XEBs[j,k] = (1/random_circ_instances)*averaged_lin_XEBs[j,k]
        #averaged_TVDs[j,k] = (1/random_circ_instances)*averaged_TVDs[j,k]
        averaged_gate_counts[j,k] = (1/random_circ_instances)*averaged_gate_counts[j,k]

averaged_mapping_depths = np.zeros((len(mappings),len(rotations)))
averaged_mapping_depths_combined = np.zeros((len(mappings),len(rotations)))
for j in range(len(mappings)):
    for k in range(len(rotations)):
        averaged_mapping_depths[j,k] = 0
        averaged_mapping_depths_combined[j,k] = 0
        for l in range(all_circ_depths.shape[1]):
            averaged_mapping_depths[j,k] += all_circ_depths[k,0,j]
            averaged_mapping_depths_combined[j,k] += all_circ_depths[k,0,j] + all_prep_depths[k,0,j]
        averaged_mapping_depths[j,k] = averaged_mapping_depths[j,k]/all_circ_depths.shape[1]
        averaged_mapping_depths_combined[j,k] = averaged_mapping_depths_combined[j,k]/all_circ_depths.shape[1]

sampling_increase = np.zeros(averaged_traces.shape)
for j in range(len(mappings)):
    for k in range(averaged_traces.shape[1]):
        #sampling_increase[j,k] = 1/(averaged_traces[j,k]*averaged_fidelities[j,k]**2)
        sampling_increase[j,k] = 1/(averaged_traces[j,k])

In [ ]:
unmitigated_averaged_fidelities = np.zeros(averaged_fidelities.shape)
for j in range(averaged_fidelities.shape[0]):
    for k in range(averaged_fidelities.shape[1]):
        unmitigated_averaged_fidelities[j,k] = averaged_traces[j,k]*averaged_fidelities[j,k]

In [ ]:
# Defined labels for each mapping
labels ={
    'jw_9': r'JW $[[9,9,1]]$',
    'lw_9': r'Compact $[[11,9,1]]$',
    'gse_9': r'GSE $[[14,8,1]]$'
}

colors = {}
for j in range(len(mappings)):
    colors[mappings[j]] = list(plt.rcParams['axes.prop_cycle'])[j]['color']

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, sharex=True,
                                    figsize=(12, 6),dpi=300)
fig.subplots_adjust(wspace = 0.25)
for j in range(len(mappings)):
    ax0.plot(rotations,averaged_mapping_depths_combined[j,:],label = labels[mappings[j]],marker='+',color=colors[mappings[j]]) #
ax0.set_xlabel(f'logical rotations')
ax0.set_ylabel(f'depth')
ax0.xaxis.label.set_color('black')
ax0.yaxis.label.set_color('black')
ax0.tick_params(axis='x', colors='black')
ax0.tick_params(axis='y', colors='black')
#ax0.margins(x=0)

for j in range(len(mappings)):
    ax1.plot(rotations,averaged_fidelities[j,:],label = labels[mappings[j]],marker='+',color=colors[mappings[j]])
ax1.set_xlabel(f'logical rotations')
ax1.set_ylabel(f'codespace fidelity')
ax1.set_yscale('log')
#ax1.ticklabel_format(axis='y',style='plain')
ax1.xaxis.label.set_color('black')
ax1.yaxis.label.set_color('black')
ax1.tick_params(axis='x', colors='black')
ax1.tick_params(axis='y', colors='black')

for j in range(len(mappings)):
    ax2.plot(rotations,sampling_increase[j,:],label = labels[mappings[j]],marker='+',color=colors[mappings[j]])
ax2.set_xlabel(f'logical rotations')
ax2.set_ylabel(f'sampling increase factor')
ax2.set_yscale('log')
ax2.xaxis.label.set_color('black')
ax2.yaxis.label.set_color('black')
ax2.tick_params(axis='x', colors='black')
ax2.tick_params(axis='y', colors='black')

legend = plt.legend(loc='upper center', bbox_to_anchor=(-.78,1.081), ncol=len(mappings), fancybox=True, shadow=False)
frame = legend.get_frame()
frame.set_facecolor('white')
frame.set_edgecolor('white')
plt.show()
#plt.savefig('9_modes_random_circuits_figure.png',facecolor='white')

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, sharex=True,
                                    figsize=(12, 4),dpi=1000)
fig.subplots_adjust(wspace = 0.25)
for j in range(len(mappings)):
    ax0.plot(rotations,averaged_mapping_depths_combined[j,:],label = labels[mappings[j]],marker='+',color=colors[mappings[j]]) #
ax0.set_xlabel(f'logical rotations')
ax0.set_ylabel(f'depth')
ax0.xaxis.label.set_color('black')
ax0.yaxis.label.set_color('black')
ax0.tick_params(axis='x', colors='black')
ax0.tick_params(axis='y', colors='black')
#ax0.margins(x=0)

for j in range(1):
    ax1.plot(rotations,averaged_fidelities[j,:],label = labels[mappings[j]],marker='+',color=colors[mappings[j]],linestyle='dashed')
for j in range(1,len(mappings)):
    ax1.plot(rotations,averaged_fidelities[j,:],label = labels[mappings[j]],marker='+',color=colors[mappings[j]])
    ax1.plot(rotations,unmitigated_averaged_fidelities[j,:],color = colors[mappings[j]],marker='+',linestyle='dashed')
ax1.set_xlabel(f'logical rotations')
ax1.set_ylabel(f'codespace fidelity')
ax1.set_yscale('log')
ax1.xaxis.label.set_color('black')
ax1.yaxis.label.set_color('black')
ax1.tick_params(axis='x', colors='black')
ax1.tick_params(axis='y', colors='black')
dummy1, = ax1.plot([],color='black',label='mitigated',marker='+')
dummy2, = ax1.plot([],color='black',linestyle='dashed',label='unmitigated',marker='+')
dummy1.set_color('gray')
dummy2.set_color('gray')
ax1.legend(handles=[dummy1,dummy2])

for j in range(len(mappings)):
    ax2.plot(rotations,sampling_increase[j,:],label = labels[mappings[j]],marker='+',color=colors[mappings[j]])
ax2.set_xlabel(f'logical rotations')
ax2.set_ylabel(f'sampling increase factor')
ax2.set_yscale('log')
ax2.xaxis.label.set_color('black')
ax2.yaxis.label.set_color('black')
ax2.tick_params(axis='x', colors='black')
ax2.tick_params(axis='y', colors='black')

legend = plt.legend(loc='upper center', bbox_to_anchor=(-.78,1.11), ncol=len(mappings), fancybox=True, shadow=False)
frame = legend.get_frame()
frame.set_facecolor('white')
frame.set_edgecolor('white')
plt.show()
#plt.savefig('9_modes_random_circuits_figure.png',facecolor='white')

#### Saving data

In [ ]:
data_dict = {
    'mappings': mappings, 
    'mapping_shots':mapping_shots, 
    'rotations':rotations,
    'random_circ_instances':random_circ_instances,
    'seeds':seeds,
    'stat_noise_rate' : stat_noise_rate,
    'gate_noise_rate' : gate_noise_rate,
    'all_gate_counts':all_gate_counts,
    'averaged_fidelities':averaged_fidelities,
    'unmitigated_averaged_fidelities':unmitigated_averaged_fidelities,
    'averaged_traces':averaged_traces,
    'averaged_mapping_depths_combined': averaged_mapping_depths_combined,
    'averaged_gate_counts':averaged_gate_counts,
    'sampling_increase':sampling_increase,
    'labels': labels, 
    'colors':colors
}

#with open(f'9moderandom_circuit_data.pkl', 'wb') as f:
#   pickle.dump(data_dict,f)